In [3]:
# Credit Card Fraud Detection with Gradio App

# Step 1: Install Gradio
!pip install gradio

# Step 2: Import Libraries
import pandas as pd
import numpy as np
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Step 3: Load and Clean Dataset
df = pd.read_csv("fraud test.csv")

# Drop irrelevant columns
cols_to_drop = ['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'first', 'last',
                'street', 'city', 'state', 'job', 'dob', 'trans_num']
df = df.drop(columns=cols_to_drop)

# Step 4: Encode Categorical Features
categorical_cols = ['merchant', 'category', 'gender']
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Step 5: Define Features and Target
X = df_encoded.drop('is_fraud', axis=1)
y = df_encoded['is_fraud']

# Step 6: Scale Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 7: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

# Step 8: Train Model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Step 9: Evaluate Model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Step 10: Build Gradio Prediction Function
def predict_fraud(amt, zip_code, lat, long, city_pop, unix_time, merch_lat, merch_long,
                  merchant, category, gender):

    input_dict = {
        'amt': amt, 'zip': zip_code, 'lat': lat, 'long': long, 'city_pop': city_pop,
        'unix_time': unix_time, 'merch_lat': merch_lat, 'merch_long': merch_long,
        'merchant': merchant, 'category': category, 'gender': gender
    }

    input_df = pd.DataFrame([input_dict])
    temp_df = pd.concat([df.drop('is_fraud', axis=1), input_df], ignore_index=True)
    temp_encoded = pd.get_dummies(temp_df, columns=categorical_cols, drop_first=True)
    temp_encoded = temp_encoded.reindex(columns=X.columns, fill_value=0)
    scaled_input = scaler.transform(temp_encoded.tail(1))
    prediction = model.predict(scaled_input)[0]
    return "Fraudulent Transaction" if prediction == 1 else "Not Fraudulent"

# Step 11: Create Gradio Interface
inputs = [
    gr.Number(label="Transaction Amount"),
    gr.Number(label="ZIP Code"),
    gr.Number(label="Latitude"),
    gr.Number(label="Longitude"),
    gr.Number(label="City Population"),
    gr.Number(label="Unix Time"),
    gr.Number(label="Merchant Latitude"),
    gr.Number(label="Merchant Longitude"),
    gr.Textbox(label="Merchant Name"),
    gr.Dropdown(df['category'].unique().tolist(), label="Transaction Category"),
    gr.Dropdown(df['gender'].unique().tolist(), label="Cardholder Gender")
]

# Launch the app
gr.Interface(fn=predict_fraud, inputs=inputs, outputs="text",
             title="Credit Card Fraud Detection").launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.9 MB/s eta 0:00:00
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12

    accuracy                           1.00        12
   macro avg       1.00      1.00      1.00        12
weighted avg       1.00      1.00      1.00        12

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detec